# Title
[]()

In [2]:
from pandas import json_normalize  
import pandas as pd
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\src")
from silvhua import *
from datetime import datetime
# import json
# from plotly.subplots import make_subplots
# import requests

In [5]:
# set the option to wrap text within cells
pd.set_option('display.max_colwidth', None)
# pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [16]:
# from pypdf import PdfReader
import pandas as pd
import openai
import os
# from openai.embeddings_utils import get_embedding, cosine_similarity
import re

In [17]:

sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from summarization import Chatbot, reply, batch_reply
from article_processing import create_text_dict

## Create relevance_prompts_df

In [20]:
from itertools import product
prompts_dict = dict()
experiment_num = 1
relevance_prompts_dict = dict()

prep_step = [
    # "Take the key points to",
    "Take the key points and numerical descriptors to",
    # ""
]

task_part1 = [
    # "Summarize the article in under 300 characters",
    "Summarize for a LinkedIn post",
    # "Summarize for a tweet",
    # "Summarize in an engaging way",
    "Describe the interesting points to your coworker at the water cooler"
    # "Summarize the article for a Tiktok post"
]
audience = [
    # "lay audience",
    # "",
    "seniors",
    "people who enjoy sports"
]
task_part2 = [
    "",
    "Use terms a 12-year-old can understand.",
    # "Assume your audience has no science background."
    # "Include the most interesting findings.",
    # "Include the key take-aways for the reader.",
    # "Include the implications of the article."
]

task_part3 =[
    "Add 1-2 sentences to make this relevant for"
    # "Add 1-2 sentences to make this relevant for older adults."
    # "Once you are done, add 1-2 sentences to make this relevant for older adults.",
    ""
]

# prompts_df = pd.DataFrame(product(prep_step, task_part1, task_part2, task_part3, audience), 
#     columns=['prep_step', 'task part 1', 'task part 2', 'task part 3', 'audience'])

prompts_df = pd.DataFrame(product(prep_step, task_part1), 
    columns=['prep_step', 'task part 1'])

relevance_prompts_df = pd.DataFrame(product(task_part3, audience), 
    columns=['task part 3', 'audience'])



prompts_df['prompt'] = prompts_df.apply(
    lambda row: f"{row['prep_step']} {row['task part 1']}.", 
    axis=1)
# prompts_df['simplify'] = prompts_df.apply(
#     lambda row: f" {row['task part 2'] if row['task part 2'] else ''}", 
#     axis=1)
relevance_prompts_df['relevance'] = relevance_prompts_df.apply(
    lambda row: f" {row['task part 3']} {row['audience']} " if row['audience'] else '', 
    axis=1) 
relevance_prompts_dict[experiment_num] = relevance_prompts_df
relevance_prompts_df

,task part 3,audience,relevance
0,Add 1-2 sentences to make this relevant for,seniors,Add 1-2 sentences to make this relevant for seniors
1,Add 1-2 sentences to make this relevant for,people who enjoy sports,Add 1-2 sentences to make this relevant for people who enjoy sports


In [21]:
prompts_df

,prep_step,task part 1,prompt
0,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post.
1,Take the key points and numerical descriptors to,Describe the interesting points to your coworker at the water cooler,Take the key points and numerical descriptors to Describe the interesting points to your coworker at the water cooler.


# Load previous dataframe 

In [22]:
filename = 'prompt_chain_experiment_2023-04-14_0708.sav'
path = r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
df = loadpickle(filename, path)
df

Time completed: 2023-04-16 15:11:26.411115


,0,1,2,3,4,5
article_title,Daily Energy Expenditure through the Human Life Course,Daily Energy Expenditure through the Human Life Course,High doses of anti-inflammatory drugs compromise muscle strength and hypertrophic adaptations to resistance training in young adults.,High doses of anti-inflammatory drugs compromise muscle strength and hypertrophic adaptations to resistance training in young adults.,Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women,Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women
system_role,You are an expert at science communication.,You are an expert at science communication.,You are an expert at science communication.,You are an expert at science communication.,You are an expert at science communication.,You are an expert at science communication.
prompt,Take the key points and numerical descriptors to Summarize for a LinkedIn post.,Take the key points and numerical descriptors to Describe the interesting points to your coworker at the water cooler.,Take the key points and numerical descriptors to Summarize for a LinkedIn post.,Take the key points and numerical descriptors to Describe the interesting points to your coworker at the water cooler.,Take the key points and numerical descriptors to Summarize for a LinkedIn post.,Take the key points and numerical descriptors to Describe the interesting points to your coworker at the water cooler.
model,gpt-3.5-turbo,gpt-3.5-turbo,gpt-3.5-turbo,gpt-3.5-turbo,gpt-3.5-turbo,gpt-3.5-turbo
response_01,"""New research reveals major changes in metabolic rate across the human life course. Adjusted expenditures for pregnant mothers and neonates remain stable at ~100%, while total and basal expenditures accelerate rapidly over the first year and progressively decline thereafter, reaching adult levels at ~20 years. Elevated tissue-specific metabolism in early life may be related to growth or development, while reduced expenditures in later life may reflect a decline in organ level metabolism. There is considerable metabolic variation among individuals, with TEE and its components varying more than ±20% even when controlling for body composition, sex, and age. These findings have implications for understanding energy balance, nutritional needs, body weight, and potential clinical investigations into disease, pharmaceutical activity, and healing."" #metabolism #lifecourse #healthresearch","Hey, did you know that a recent study investigated daily energy expenditure throughout the human life course? They found that there are major changes in metabolic rate across our lifespan. For instance, after rapid acceleration in total and basal expenditure during the first year, adjusted expenditures progressively decline thereafter, reaching adult levels at around 20 years old. Additionally, they found that tissue-specific metabolic rates are elevated in children and adolescents by around 50% compared to adults, and are about 25% lower in the oldest subjects. Furthermore, there is considerable metabolic variation among individuals, with total energy expenditure varying more than ± 20% even when controlling for fat-free mass, fat mass, sex, and age. These findings could have implications for understanding health and disease.",Maximal over-the-counter doses of NSAIDs compromise resistance exercise-induced increases in muscle size and strength in young adults. Ibuprofen was found to have a marked effect on muscle hypertrophy and a reduction in IL-6 gene expression. The study highlights the importance of drug dosage in resistance training studies and raises concerns about the use of NSAIDs during exercise. #NSAIDs #resistancetraining #musclestrength #musclehypertrophy #exerciseadaptations,"Hey, did you know that high doses of anti-inflammatory drugs like ibuprofen can compromise muscle strength and hypertrophic adaptations to resistance training in young adults? A study showed that the maximal over-the-counter doses of ibuprof

In [27]:
df[[4,5]].head(10)

4  \
article_title                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

# Debug CSV text encoding

In [4]:
df.loc['response_04', 3]

'Hey, did you know that a recent study found that high doses of anti-inflammatory drugs, specifically ibuprofen, can compromise muscle strength and hypertrophic adaptations to resistance training in young adults? The study showed that maximal over-the-counter doses of ibuprofen can reduce muscle hypertrophy by 7.5% and attenuate increases in muscle strength, especially when training is performed with maximal all-out repetitions. The study also found that the negative effect of ibuprofen on muscle adaptations to resistance exercise is dependent on the dose of intake, and that the drug can interfere with the normal hypertrophic development of 0.1\x960.2% per day seen in other studies employing conventional resistance-training programmes for the knee extensors. The study provides strong proof-of-principle evidence that the effect of NSAIDs on muscle adaptations to resistance exercise is dependent on the dose of intake, and the mechanistic findings should contribute to a better understandi

In [11]:
df.head(8)[[2,3]]

,2,3
article_title,High doses of anti-inflammatory drugs compromise muscle strength and hypertrophic adaptations to resistance training in young adults.,High doses of anti-inflammatory drugs compromise muscle strength and hypertrophic adaptations to resistance training in young adults.
system_role,You are an expert at science communication.,You are an expert at science communication.
prompt,Take the key points and numerical descriptors to Summarize for a LinkedIn post.,Take the key points and numerical descriptors to Describe the interesting points to your coworker at the water cooler.
model,gpt-3.5-turbo,gpt-3.5-turbo
response_01,Maximal over-the-counter doses of NSAIDs compromise resistance exercise-induced increases in muscle size and strength in young adults. Ibuprofen was found to have a marked effect on muscle hypertrophy and a reduction in IL-6 gene expression. The study highlights the importance of drug dosage in resistance training studies and raises concerns about the use of NSAIDs during exercise. #NSAIDs #resistancetraining #musclestrength #musclehypertrophy #exerciseadaptations,"Hey, did you know that high doses of anti-inflammatory drugs like ibuprofen can compromise muscle strength and hypertrophic adaptations to resistance training in young adults? A study showed that the maximal over-the-counter doses of ibuprofen compromised muscle hypertrophy independent of training mode, and increased muscle strength was attenuated by ibuprofen only when training was performed with maximal all-out repetitions. The study found that the negative effect of NSAIDs on muscle adaptations to resistance exercise is dependent on the dose of intake, and there may be distinct differences in the response to resistance training and NSAID consumption in the young vs. the elderly. This study provides strong proof-of-principle evidence that the effect of NSAIDs on muscle adaptations to resistance exercise is dependent on the dose of intake."
response_02,"High doses of anti-inflammatory drugs, specifically ibuprofen, can compromise muscle strength and hypertrophic adaptations to resistance training in young adults. This was shown in a study where healthy young men and women performed 8 weeks of supervised resistance training with concomitant high- or low-dose NSAID treatment. The study found that maximal doses of ibuprofen compromised resistance exercise-induced muscle hypertrophy independent of training mode, and increases in muscle strength were attenuated by ibuprofen only when training was performed with maximal all-out repetitions. This study highlights the importance of drug dosage when taking NSAIDs and their effect on muscle adaptations to resistance exercise.","Hey, did you know that high doses of anti-inflammatory drugs like ibuprofen can compromise muscle strength and hypertrophic adaptations to resistance training in young adults? In a recent study, healthy young men and women performed 8 weeks of supervised resistance training with concomitant high- or low-dose NSAID treatment. The major findings were that maximal over-the-counter doses of ibuprofen compromised resistance exercise-induced muscle hypertrophy independent of training mode and increases in muscle strength were attenuated by ibuprofen only when training was performed with maximal all-out repetitions. The increase in muscle volume in the ASA group was of a similar magnitude to that previously reported by other studies, while the reduction in muscle hypertrophy in response to the ibuprofen treatment was quite remarkable. The study shows that daily consumption of high doses of NSAIDs attenuates strength and muscle hypertrophic adaptations to resistance training in young healthy individuals, and the effect of NSAIDs on muscle adaptations to resistance exercise is dependent on the dose of intake."
response_03,"High doses of anti-inflammatory drugs, specifically ibuprofen, compromise muscle strength and hypertrophic adaptations to resistance training in young adults, acco

In [12]:
save_filename = '../output/CSV/2023-04-16 1450.csv'
df.head(8)[[2,3]].to_csv(save_filename, encoding='utf-8', index=False)


In [13]:
df.dtypes

0    object
1    object
2    object
3    object
4    object
5    object
dtype: object

## Unicode Excape

In [23]:
# df2 = df.head(8)[[2,3]].apply(lambda x: x.encode().decode('unicode_escape'))
df2 = df.head(8)[[2,3]]

In [24]:
df2

,2,3
article_title,High doses of anti-inflammatory drugs compromise muscle strength and hypertrophic adaptations to resistance training in young adults.,High doses of anti-inflammatory drugs compromise muscle strength and hypertrophic adaptations to resistance training in young adults.
system_role,You are an expert at science communication.,You are an expert at science communication.
prompt,Take the key points and numerical descriptors to Summarize for a LinkedIn post.,Take the key points and numerical descriptors to Describe the interesting points to your coworker at the water cooler.
model,gpt-3.5-turbo,gpt-3.5-turbo
response_01,Maximal over-the-counter doses of NSAIDs compromise resistance exercise-induced increases in muscle size and strength in young adults. Ibuprofen was found to have a marked effect on muscle hypertrophy and a reduction in IL-6 gene expression. The study highlights the importance of drug dosage in resistance training studies and raises concerns about the use of NSAIDs during exercise. #NSAIDs #resistancetraining #musclestrength #musclehypertrophy #exerciseadaptations,"Hey, did you know that high doses of anti-inflammatory drugs like ibuprofen can compromise muscle strength and hypertrophic adaptations to resistance training in young adults? A study showed that the maximal over-the-counter doses of ibuprofen compromised muscle hypertrophy independent of training mode, and increased muscle strength was attenuated by ibuprofen only when training was performed with maximal all-out repetitions. The study found that the negative effect of NSAIDs on muscle adaptations to resistance exercise is dependent on the dose of intake, and there may be distinct differences in the response to resistance training and NSAID consumption in the young vs. the elderly. This study provides strong proof-of-principle evidence that the effect of NSAIDs on muscle adaptations to resistance exercise is dependent on the dose of intake."
response_02,"High doses of anti-inflammatory drugs, specifically ibuprofen, can compromise muscle strength and hypertrophic adaptations to resistance training in young adults. This was shown in a study where healthy young men and women performed 8 weeks of supervised resistance training with concomitant high- or low-dose NSAID treatment. The study found that maximal doses of ibuprofen compromised resistance exercise-induced muscle hypertrophy independent of training mode, and increases in muscle strength were attenuated by ibuprofen only when training was performed with maximal all-out repetitions. This study highlights the importance of drug dosage when taking NSAIDs and their effect on muscle adaptations to resistance exercise.","Hey, did you know that high doses of anti-inflammatory drugs like ibuprofen can compromise muscle strength and hypertrophic adaptations to resistance training in young adults? In a recent study, healthy young men and women performed 8 weeks of supervised resistance training with concomitant high- or low-dose NSAID treatment. The major findings were that maximal over-the-counter doses of ibuprofen compromised resistance exercise-induced muscle hypertrophy independent of training mode and increases in muscle strength were attenuated by ibuprofen only when training was performed with maximal all-out repetitions. The increase in muscle volume in the ASA group was of a similar magnitude to that previously reported by other studies, while the reduction in muscle hypertrophy in response to the ibuprofen treatment was quite remarkable. The study shows that daily consumption of high doses of NSAIDs attenuates strength and muscle hypertrophic adaptations to resistance training in young healthy individuals, and the effect of NSAIDs on muscle adaptations to resistance exercise is dependent on the dose of intake."
response_03,"High doses of anti-inflammatory drugs, specifically ibuprofen, compromise muscle strength and hypertrophic adaptations to resistance training in young adults, acco

In [25]:
# Define decoding function
def decode_string(s):
    return s.encode().decode('unicode_escape')

# Apply decoding function to every element of the dataframe
df2 = df2.applymap(decode_string)

df2.loc['response_04', 3]


'Hey, did you know that a recent study found that high doses of anti-inflammatory drugs, specifically ibuprofen, can compromise muscle strength and hypertrophic adaptations to resistance training in young adults? The study showed that maximal over-the-counter doses of ibuprofen can reduce muscle hypertrophy by 7.5% and attenuate increases in muscle strength, especially when training is performed with maximal all-out repetitions. The study also found that the negative effect of ibuprofen on muscle adaptations to resistance exercise is dependent on the dose of intake, and that the drug can interfere with the normal hypertrophic development of 0.1Â\x960.2% per day seen in other studies employing conventional resistance-training programmes for the knee extensors. The study provides strong proof-of-principle evidence that the effect of NSAIDs on muscle adaptations to resistance exercise is dependent on the dose of intake, and the mechanistic findings should contribute to a better understand

In [26]:
save_filename = '../output/CSV/2023-04-16 1512.csv'
df2.to_csv(save_filename, index=False)

In [ ]:
# Define decoding function
def decode_string(s):
    return s.encode().decode('unicode_escape')

# Apply decoding function to every element of the dataframe
df2 = df2.applymap(decode_string)

df2.loc['response_04', 3]


'Hey, did you know that a recent study found that high doses of anti-inflammatory drugs, specifically ibuprofen, can compromise muscle strength and hypertrophic adaptations to resistance training in young adults? The study showed that maximal over-the-counter doses of ibuprofen can reduce muscle hypertrophy by 7.5% and attenuate increases in muscle strength, especially when training is performed with maximal all-out repetitions. The study also found that the negative effect of ibuprofen on muscle adaptations to resistance exercise is dependent on the dose of intake, and that the drug can interfere with the normal hypertrophic development of 0.1Â\x960.2% per day seen in other studies employing conventional resistance-training programmes for the knee extensors. The study provides strong proof-of-principle evidence that the effect of NSAIDs on muscle adaptations to resistance exercise is dependent on the dose of intake, and the mechanistic findings should contribute to a better understand

# Iteration 2

# *End of Page*